### Analyze dataset processed in clean_BB_data.ipynb
- Data file is in Cleaned_Trips2016_10.csv
- Read data as feather format and load to pandas dataframe
- We want to predict NextLegBunchingFlag, NextNextLegBunchingFlag, NextNextNextLegBunchingFlag  
- Data is highly imbalanced. 
- Careful in choosing train test and validation data 
- Do not want to use random subset of data
- Try stratified sampling.
- Data has 138752 rows, considering it as medium dataset choose test:10%, validation:10% and train:80%.
- Grid search for hyper parameter tuning.
- Random search for hyperparameter tuning.
- Saved trained models using joblib as 'Random_forest_optimized.joblib' and 'Random_forest_not_optimized.joblib'
- Permutation importance for feature importance.
- Using 54 original features obtained mcc_score of 0.8550, training time 34 minute.
- Selecting top 8 features obtained mcc_score of 0.8577, training time 81 s.
- Logistic Regression, used feature scalling.
- Looked at threshold precision recall curve.
- Wrote stuff in class


In [1]:
import sys
import os

current_path = os.path.abspath(os.path.join('..'))
if current_path not in sys.path:
    sys.path.append(current_path)
    
from random import randint
import pandas as pd
import numpy as np
from helper.imports import *
from helper.structured  import *
from IPython.display import display
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from helper.plots_and_scores import *
%matplotlib inline

/Applications/anaconda3/envs/DNN/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# Load the cleaned dataset 
data = pd.read_feather(f'{current_path}/data/processed/Cleaned_Trips2016_10.csv')

In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [ ]:
display_all(data.head())

In [ ]:
display_all(data.dtypes)

In [ ]:
data = data.drop(['index'],axis =1)

In [ ]:
data.info()

In [ ]:
data['NextLegBunchingFlag'].value_counts()

In [ ]:
data.NextLegBunchingFlag.hist()

In [ ]:
data.NextNextLegBunchingFlag.hist()

In [ ]:
data.NextNextNextLegBunchingFlag.hist()

In [ ]:
# Convert all columns to numeric column and separate out the target variable
X, y, nas  = proc_df(data, y_fld ='NextLegBunchingFlag',skip_flds=['NextNextLegBunchingFlag',\
                     'NextNextNextLegBunchingFlag'])

In [ ]:
X.columns

In [ ]:
X.dtypes

### Stratified sampling for train, test and validation datasets

In [ ]:
# Shuffle and split the data into training and testing subsets
X_train, X_test_valid, y_train, y_test_valid = train_test_split(X, y,  random_state =1, \
                                                                test_size=0.20,stratify = y)
X_test, X_valid, y_test, y_valid = train_test_split(X_test_valid, y_test_valid,  random_state =1, \
                                                                test_size=0.50,stratify = y_test_valid)

In [ ]:
#Look at distribution of targets in train, test and validation set
print('train:',np.unique(y_train,return_counts=True))
print('test:',np.unique(y_test,return_counts=True))
print('valid:',np.unique(y_valid,return_counts=True))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
%time RFmodel = RandomForestClassifier(random_state=0,n_estimators=40,criterion='gini',\
                                       n_jobs=-1,oob_score=True)
%time RFmodel.fit(X_train,y_train)
#print_scores(RFmodel)

In [ ]:
y_pred= RFmodel.predict(X_train)
y_pred_proba = RFmodel.predict_proba(X_train)

In [ ]:
ps = plot_and_scores(y_train,y_pred, y_pred_proba)

In [ ]:
ps.precision_recall_vs_threshold()

In [ ]:
precisions, recalls, thresholds = precision_recall_curve(y_train,y_pred_proba[:,1])
plt.figure()
plt.title("Precision-Recall vs Threshold Chart")
plt.plot(thresholds, precisions[: -1], "b--", label="Precision")
plt.plot(thresholds, recalls[: -1], "r--", label="Recall")
plt.ylabel("Precision, Recall")
plt.xlabel("Threshold")
plt.legend(loc="lower left")
        #plt.ylim([0,1])
plt.show() 

In [ ]:
ps.display_confusion_matrix()

### Grid search parameters then try Random Search

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import fbeta_score
from pprint import pprint

In [ ]:
RFmodel = RandomForestClassifier(random_state=0,criterion='gini',\
                                       n_jobs=-1,oob_score=True,n_estimators=100)
# Create the parameters list to tune, using a dictionary if needed.
parameters = {'min_samples_leaf': [2,3],'max_features':[0.3,0.5],\
                    'max_samples':[0.6,0.8,1]}
# Make an mcc_Metric scoring object using make_scorer()
scorer = make_scorer(mcc_Metric, greater_is_better=True)
# Perform grid search on the classifier using 'scorer' as the scoring method using GridSearchCV()
print("In instantiate now ....")
%time grid_obj = GridSearchCV(RFmodel, parameters, scoring = scorer)
# Fit the grid search object to the training data and find the optimal parameters using fit()
print("In fit now ....")
%time grid_fit = grid_obj.fit(X_train, y_train)
# Get the estimator
best_clf = grid_fit.best_estimator_
# Make predictions using the unoptimized model
print("In unoptimized fit now ....")
%time Unoptimizedpredictions = (RFmodel.fit(X_train, y_train)).predict(X_test)

# Make predictions using the optimized and model
best_predictions = best_clf.predict(X_test)
print('Results--------------\n')
# Report the before-and-afterscores
print("Unoptimized model\n------")
print("MCC score on testing data: {:.4f}".format(mcc_Metric(y_test, Unoptimizedpredictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, Unoptimizedpredictions, beta = 0.5)))
print("\nOptimized Model\n------")
print("Final MCC score on the testing data: {:.4f}".format(mcc_Metric(y_test, best_predictions)))
print("Final F-score on the testing data: {:.4f}".format(fbeta_score(y_test, best_predictions, beta = 0.5)))
best_clf

### ROC Curves

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, auc
# Compute ROC curve and ROC area for each class
def plot_ROC(y_test, best_predictions):
    n_classes = 1
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test, best_predictions)
        roc_auc[i] = auc(fpr[i], tpr[i])
# Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), best_predictions.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    plt.figure()
    lw = 2
    plt.plot(fpr[0], tpr[0], color='darkorange',
         lw=lw, label='ROC curve (area = %0.4f)' % roc_auc[0])
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic for optimized model')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
plot_ROC(y_test, best_predictions)

In [ ]:
%time un_opt_model = RFmodel.fit(X_train, y_train)
Unopt_pred_train =un_opt_model.predict(X_train)

### Random Search parameters

In [ ]:
from scipy.stats import truncnorm, uniform
from random import randint

# create random forest classifier model
rf_model = RandomForestClassifier(n_jobs=-1)
model_params = dict(
    # randomly sample numbers from 4 to 204 estimators
    n_estimators=[randint(100,250)],
    #n_estimators =[80,100],
    #normally distributed max_features, with mean .25 stddev 0.1, bounded between 0 and 1
    #max_features=truncnorm(a=0, b=1, loc=0.25, scale=0.1),
    max_features = [0.3,0.5],
    # uniform distribution from 0.01 to 0.2 (0.01 + 0.199)
    #min_samples_split=uniform(0.01, 0.399),
    #uniform distribution from 0.5 to 0.99 (0.5 + 0.99)
    #max_samples=uniform(0.5,0.49),
    max_samples= [0.8,0.9],
    min_samples_leaf=[2,3,5]
)
#Set up the random search estimator, this will train 5 models.
clf = RandomizedSearchCV(rf_model, model_params, n_iter = 10,random_state=0,scoring = scorer)
print("In fit now")
%time model_rs = clf.fit(X_train, y_train)
# Get the best estimator
best_clf_rs = model_rs.best_estimator_
#Make predictions using the unoptimized
#predictions_UO = (RFmodel.fit(X_train, y_train)).predict(X_test)
#Make predictions using the optimized
best_pred_rs_train = best_clf_rs.predict(X_train)
best_pred_rs = best_clf_rs.predict(X_test)

# Report the before-and-afterscores
print('Results--------------\n')
print("Unoptimized model\n------")
print("MCC score on training data: {:.4f}".format(mcc_Metric(y_train, Unopt_pred_train)))
print("MCC score on testing data: {:.4f}".format(mcc_Metric(y_test, Unoptimizedpredictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, Unoptimizedpredictions, beta = 0.5)))
print("\nOptimized Model\n------")
print("Final MCC score on the testing data: {:.4f}".format(mcc_Metric(y_train, best_pred_rs_train)))
print("Final MCC score on the testing data: {:.4f}".format(mcc_Metric(y_test, best_pred_rs)))
print("Final F-score on the testing data: {:.4f}".format(fbeta_score(y_test, best_pred_rs, beta = 0.5)))
# Print the best parameters
pprint(model_rs.best_estimator_.get_params())

In [ ]:
clf.scorer_

In [ ]:
plot_ROC(y_test,best_predictions_rs)

#### Saving the models
- Unoptimized model. 
- Optimized model.

In [ ]:
import joblib

In [ ]:
joblib.dump(model_rs,'Random_forest_optimized.joblib')
joblib.dump(RFmodel,'Random_forest_not_optimized.joblib')

In [ ]:
load_RF = joblib.load('Random_forest.joblib')

In [ ]:
y_pd = load_RF.predict(X_test)
print("Final MCC score on the testing data: {:.4f}".format(mcc_Metric(y_test, y_pd)))

#### Feature importance.
- Use permutation importance on the test set

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
result = permutation_importance(load_RF, X_test, y_test, n_repeats=10,random_state=0)
sorted_idx = result.importances_mean.argsort()
fig, ax = plt.subplots(figsize=(12, 8))
ax.boxplot(result.importances[sorted_idx].T,
           vert=False, labels=X_test.columns[sorted_idx])
ax.set_title("Permutation Importances (test set)")
fig.tight_layout()
plt.show()

### Choose top 8 features . For the rest of the analysis we will use this features for modelling 

In [ ]:
X_test.columns[sorted_idx[-8:]]

In [ ]:
X_test_features = X_test[['Temp', 'DwellTime', 'OnsAndOffsCompensated', 'ArriveLoadCompensated',\
       'ArriveDelay', 'StopName', 'HeadwayOffset', 'ActualHeadway']]

X_train_features = X_train[['Temp', 'DwellTime', 'OnsAndOffsCompensated', 'ArriveLoadCompensated',\
       'ArriveDelay', 'StopName', 'HeadwayOffset', 'ActualHeadway']]

X_valid_features = X_valid[['Temp', 'DwellTime', 'OnsAndOffsCompensated', 'ArriveLoadCompensated',\
       'ArriveDelay', 'StopName', 'HeadwayOffset', 'ActualHeadway']]

### Train model on the top 8 features

In [ ]:
import time
def RandomtuneRF(X_tr, y_tr, X_test, y_test):
    # This returns best_pred_train, best_pred_test, model
    # create random forest classifier model
    model = RandomForestClassifier(n_jobs=-1)
    model_params = dict(
    # randomly sample numbers from 4 to 204 estimators
        n_estimators=[randint(100,200)],
    #n_estimators =[80,100],
    #normally distributed max_features, with mean .25 stddev 0.1, bounded between 0 and 1
    #max_features=truncnorm(a=0, b=1, loc=0.25, scale=0.1),
        max_features = [0.3,0.5],
    # uniform distribution from 0.01 to 0.2 (0.01 + 0.199)
    #min_samples_split=uniform(0.01, 0.399),
    #uniform distribution from 0.5 to 0.99 (0.5 + 0.99)
    #max_samples=uniform(0.5,0.49),
        max_samples= [0.8,0.9],
        min_samples_leaf=[2,3,5]
    )
#Set up the random search estimator, this will train 5 models.
    clf = RandomizedSearchCV(model, model_params, n_iter = 3,random_state=0,scoring = scorer)
    print("In fit now--------\n")
    t = time.process_time()
    fit_model = clf.fit(X_tr, y_tr)
    elapsed_time = time.process_time() - t
    print("Time to fit: {:.4f} s".format(elapsed_time))
# Get the best estimator
    best_clf = fit_model.best_estimator_
#Make predictions using the unoptimized
#predictions_UO = (RFmodel.fit(X_train, y_train)).predict(X_test)
#Make predictions using the optimized
    best_pred_train = best_clf.predict(X_tr)
    best_pred_test = best_clf.predict(X_test)
    return fit_model,best_pred_train, best_pred_test
    # Print the best parameters
    pprint(model_rs.best_estimator_.get_params())
    
def compare_models(y_train,y_test,Unopt_pred_train,Unoptimizedpredictions,best_pred_train,best_pred_test):
#Report the before-and-afterscores
    print('Results--------------\n')
    print("Unoptimized model\n------")
    print("MCC score on training data: {:.4f}".format(mcc_Metric(y_train, Unopt_pred_train)))
    print("MCC score on testing data: {:.4f}".format(mcc_Metric(y_test, Unoptimizedpredictions)))
    print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, Unoptimizedpredictions, beta = 0.5)))
    print("\nOptimized Model\n------")
    print("Final MCC score on the training data: {:.4f}".format(mcc_Metric(y_train, best_pred_train)))
    print("Final MCC score on the testing data: {:.4f}".format(mcc_Metric(y_test, best_pred_test)))
    print("Final F-score on the testing data: {:.4f}".format(fbeta_score(y_test, best_pred_test, beta = 0.5)))
    plot_ROC(y_test,best_pred_test)

In [ ]:
fit_model,best_pred_train, best_pred_test = RandomtuneRF(X_train_features, y_train, X_test_features, y_test)
compare_models(y_train,y_test,Unopt_pred_train,Unoptimizedpredictions,best_pred_train,best_pred_test)

### Save the fitted model

In [ ]:
import joblib
joblib.dump(fit_model,'Random_forest_optimized_8_features.joblib')

#### Put everything in a class.

In [ ]:
class RF_model:
    """ Runs Random forest """
    def __init__(self):
        self.random_forest = RandomForestClassifier(n_jobs=-1,criterion ='gini')
    
    def unoptimized_RF(self,X, y):
    """ 
    Trains a model with default parameters and returns the model
    Parameter
    ---------
    X : input features
    y : target
    
    Returns
    --------
    un_opt_model: trained model
    
    """
        self.X = X
        self.y = y
        print("In unoptimized fit now--------")
        t = time.process_time()
        un_opt_model = self.random_forest.fit(self.X, self.y)
        elapsed_time = time.process_time() - t
        print("Time taken to fit: {:.4f} s".format(elapsed_time))
        print('\n')
        #un_opt_pred =un_opt_model.predict(self.X)
        return un_opt_model  
    
    def randomtuned_RF(self, X_train, y_train):
    """ 
    Trains a model using random parameters and returns the model
    Parameter
    ---------
    X : input features
    y : target
    
    Returns
    --------
    best_clf : trained model
    
    """
        self.X_train = X_train
        self.y_train = y_train
        # create random forest classifier model
        model = self.random_forest
        model_params = dict(
    # randomly sample numbers from 4 to 204 estimators
            n_estimators=[randint(120,200)],
    #n_estimators =[80,100],
    #normally distributed max_features, with mean .25 stddev 0.1, bounded between 0 and 1
    #max_features=truncnorm(a=0, b=1, loc=0.25, scale=0.1),
            max_features = [0.3,0.5,0.8],
    # uniform distribution from 0.01 to 0.2 (0.01 + 0.199)
    #min_samples_split=uniform(0.01, 0.399),
    #uniform distribution from 0.5 to 0.99 (0.5 + 0.99)
    #max_samples=uniform(0.5,0.49),
            max_samples= [0.8,0.9,0.99],
            min_samples_leaf=[2,3,5]
        )
#Set up the random search estimator, this will train 5 models.
        clf = RandomizedSearchCV(model, model_params, n_iter = 5,random_state=0,scoring = scorer)
        print("In randomtune fit now--------")
        t = time.process_time()
        fit_model = clf.fit(self.X_train, self.y_train)
        elapsed_time = time.process_time() - t
        print("Time taken to fit: {:.4f} s".format(elapsed_time))
        print('\n')
        # Get the best estimator
        best_clf = fit_model.best_estimator_
        #Print the best parameters
        #pprint(fit_model.best_estimator_.get_params())
        return best_clf
    

        # Compute ROC curve and ROC area for each class
    def plot_ROC(self,y,y_pred,name):
        self.y = y
        self.y_pred = y_pred
        self.name = name
        n_classes = 1
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(self.y, self.y_pred)
            roc_auc[i] = auc(fpr[i], tpr[i])
# Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(self.y.ravel(), self.y_pred.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        plt.figure()
        lw = 2
        plt.plot(fpr[0], tpr[0], color='darkorange',
             lw=lw, label='ROC curve (area = %0.4f)' % roc_auc[0])
        plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic %s'%name)
        plt.legend(loc="lower right")
        plt.show()
        
    def scores(self,y,y_pred):
        #Takes prdictions from two models and outputs the MCC score and f beta score
        self.y = y
        self.y_pred = y_pred
      #Report the before-and-afterscores
        print("Accuracy score: {:.4f}".format(accuracy_score(self.y, self.y_pred)))
        print("MCC score: {:.4f}".format(mcc_Metric(self.y,self.y_pred)))
        print("F-score: {:.4f}".format(fbeta_score(self.y, self.y_pred, beta = 0.5)))
           

In [ ]:
# Here we return the models
rf_model = RF_model()
# First train a base model with default parameters.
base_model = rf_model.unoptimized_RF(X_train_features,y_train)
# Now train a model to optimize parameters.
optimized_model = rf_model.randomtuned_RF(X_train_features,y_train)

In [ ]:
#Optimized model
y_pred_train = optimized_model.predict(X_train_features)
y_pred_test = optimized_model.predict(X_test_features)

y_pred_train_base = base_model.predict(X_train_features)
y_pred_test_base = base_model.predict(X_test_features)
#Try optimized model
print("Optimized model-------")
print("Train score:-->")
rf_model.scores(y_train,y_pred_train)
#rf_model.plot_ROC(y_train[:2000],y_pred_train,'optimized model, train')
print("\n Test score:-->")
rf_model.scores(y_test,y_pred_test)
rf_model.plot_ROC(y_test,y_pred_test,'optimized model, test')
print("Not optimized model--------")
print("Train score:-->")
#Try not optimized model
rf_model.scores(y_train,y_pred_train_base)
#rf_model.plot_ROC(y_train[:2000],y_pred_train_base,'Not optimized model, train')
print("\n Test score:-->")
rf_model.scores(y_test,y_pred_test_base)
rf_model.plot_ROC(y_test,y_pred_test_base,'Not optimized model, test')

In [ ]:
joblib.dump(optimized_model,'Random_forest_optimized_8_features.joblib')

### Now try Logisitic regression model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_curve

class log_reg_model():
    
    def __init__(self, X,y):
        self.log_reg = LogisticRegression(max_iter=1000, C=100000,penalty = 'l2')
        self.X = X
        self.y = y
        
    def unoptimized_LR(self):
       #takes in X and y and trains a model with default parameters and returns the model
        print("In unoptimized fit now--------")
        t = time.process_time()
        un_opt_model = self.log_reg.fit(self.X, self.y)
        elapsed_time = time.process_time() - t
        print("Time taken to fit: {:.4f} s".format(elapsed_time))
        print('\n')
        #un_opt_pred =un_opt_model.predict(self.X)
        return un_opt_model
    
    
class plot_and_scores():
    """ 
    Input 
    ---------
    y : actual target values
    y_pred : predicted target  values 
    y_pred_proba : predicted probabilities  
    
    """
    def __init__(self,y,y_pred, y_pred_proba):
        self.y = y
        self.y_pred = y_pred
        self.y_pred_proba = y_pred_proba
        
    def plot_ROC(self,name):
        self.name = name
        n_classes = 1
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(self.y, self.y_pred)
            roc_auc[i] = auc(fpr[i], tpr[i])
        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(self.y.ravel(), self.y_pred.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        plt.figure()
        lw = 2
        plt.plot(fpr[0], tpr[0], color='darkorange',
             lw=lw, label='ROC curve (area = %0.4f)' % roc_auc[0])
        plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic %s'%name)
        plt.legend(loc="lower right")
        plt.show()
        
    def print_scores(self):
        print("Accuracy score: {:.4f}".format(accuracy_score(self.y, self.y_pred)))
        print("MCC score: {:.4f}".format(mcc_Metric(self.y,self.y_pred)))
        print("F-score: {:.4f}".format(fbeta_score(self.y, self.y_pred, beta = 0.5)))
    
    
    def mcc_Metric(self):
        
        cf_matrix=confusion_matrix(self.y,self.y_preds)
        TP = cf_matrix[0][0]
        TN = cf_matrix[1][1]
        FN = cf_matrix[0][1]
        FP = cf_matrix[1][0]
        N = TN + TP + FN + FP 
        S = (TP + FN)/N
        P = (TP + FP)/N
        num = (TP /N) - (S*P)
        deno = np.sqrt(P*S*(1-S)*(1-P))
        
        #Need to avoide division by zero
        return weird_division(num,deno)
    
    def display_confusion_matrix(self):
        cm = confusion_matrix(self.y, self.y_pred)
        ConfusionMatrixDisplay(cm).plot() 
        

    def weird_division(self,n, d):
        self.n = n
        self.d = d
        return self.n / self.d if self.d else 0  
   
   
    def precision_recall_vs_threshold(self):
        precisions, recalls, thresholds = precision_recall_curve(self.y,self.y_pred_proba[:,1])
        plt.figure()
        plt.title("Precision-Recall vs Threshold Chart")
        plt.plot(thresholds, precisions[: -1], "b--", label="Precision")
        plt.plot(thresholds, recalls[: -1], "r--", label="Recall")
        plt.ylabel("Precision, Recall")
        plt.xlabel("Threshold")
        plt.legend(loc="lower left")
        plt.ylim([0,1])
        plt.show() 

In [ ]:
from sklearn.preprocessing import StandardScaler
X1 = StandardScaler().fit_transform(X_train_features)

In [ ]:
X1.shape

In [ ]:
# Here we return the models
log_r_model = log_reg_model(X1,y_train)
# First train a base model with default parameters.
base_model_LR = log_r_model.unoptimized_LR()

In [ ]:
#Make predictions
y_pred_LR = base_model_LR.predict(X1)
y_pred_proba = base_model_LR.predict_proba(X1)
#y_pred_test_base = base_model.predict(X_test_features)
ps = plot_and_scores(y_train,y_pred_LR,y_pred_proba)
ps.print_scores()
ps.plot_ROC('test')
ps.display_confusion_matrix()
ps.precision_recall_vs_threshold()

In [ ]:
y_pred_LR_prob = base_model_LR.predict_proba(X1)

In [ ]:
from sklearn.metrics import precision_recall_curve
precisions, recalls, thresholds = precision_recall_curve(y_train,y_pred_LR_prob[:,1])

In [ ]:
print('precision shape',precisions.shape, 'threshold shape',thresholds.shape,'recalls shape',recalls.shape)

In [ ]:
111001-110752

In [ ]:
# probs_y is a 2-D array of probability of being labeled as 0 (first column of #array) vs 1 (2nd column in array)

#precision, recall, thresholds = precision_recall_curve(y_train,y_pred_LR_prob[:,1]) 
#retrieve probability of being 1(in second column of probs_y)
#pr_auc = metrics.auc(recall, precision)
plt.title("Precision-Recall vs Threshold Chart")
plt.plot(thresholds, precisions[: -1], "b--", label="Precision")
plt.plot(thresholds, recalls[: -1], "r--", label="Recall")
plt.ylabel("Precision, Recall")
plt.xlabel("Threshold")
plt.legend(loc="lower left")
plt.ylim([0,1])

In [ ]:
plt.plot(precision,recall)

In [ ]:
def plot_precision_vs_recall(precisions, recalls):
    plt.plot(recalls, precisions, "b-", linewidth=2)
    plt.xlabel("Recall", fontsize=16)
    plt.ylabel("Precision", fontsize=16)
    #plt.axis([0, 1, 0, 1])
    plt.grid(True)

plt.figure(figsize=(8, 6))
plot_precision_vs_recall(precisions, recalls)
#plt.plot([0.4368, 0.4368], [0., 0.9], "r:")
#plt.plot([0.0, 0.4368], [0.9, 0.9], "r:")
#plt.plot([0.4368], [0.9], "ro")
#save_fig("precision_vs_recall_plot")
plt.show()

In [ ]:
#Plot scatter matrix for all features
#params = ["HeadwayOffset","Temp", "Windspeed","Dwell Time"]
#pd.scatter_matrix(X_train, alpha = 0.3, figsize = (14,8), diagonal = 'kde');
#plt.savefig("scatter.png")

In [ ]:
from sklearn.metrics import fbeta_score, accuracy_score
def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    results = {}
    
    # TODO: Fit the learner to the training data using slicing with 'sample_size' using .fit(training_features[:], training_labels[:])
    start = time() # Get start time
    learner = learner.fit(X_train[:sample_size],y_train[:sample_size])
    end = time() # Get end time
    
    # TODO: Calculate the training time
    results['train_time'] = end - start
        
    # TODO: Get the predictions on the test set(X_test),
    #       then get predictions on the first 300 training samples(X_train) using .predict()
    start = time() # Get start time
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train[:300])
    end = time() # Get end time
    
    # TODO: Calculate the total prediction time
    results['pred_time'] = end - start
            
    # TODO: Compute accuracy on the first 300 training samples which is y_train[:300]
    results['acc_train'] = accuracy_score(y_train[:300], predictions_train[:300])
        
    # TODO: Compute accuracy on test set using accuracy_score()
    results['acc_test'] = accuracy_score(y_test, predictions_test)
    
    # TODO: Compute F-score on the the first 300 training samples using fbeta_score()
    results['f_train'] = fbeta_score(y_train[:300], predictions_train, beta=0.5)
        
    # TODO: Compute F-score on the test set which is y_test
    results['f_test'] = fbeta_score(y_test, predictions_test, beta=0.5)
       
    # Success
    print("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
        
    # Return the results
    return results

In [ ]:
import visuals as vs

In [ ]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
import time

# TODO: Initialize the three models
clf_A = LogisticRegression(random_state=0)
clf_B = DecisionTreeClassifier(random_state=0)
clf_C = AdaBoostClassifier(random_state=0)

# TODO: Calculate the number of samples for 1%, 10%, and 100% of the training data
# HINT: samples_100 is the entire training set i.e. len(y_train)
# HINT: samples_10 is 10% of samples_100 (ensure to set the count of the values to be `int` and not `float`)
# HINT: samples_1 is 1% of samples_100 (ensure to set the count of the values to be `int` and not `float`)
samples_100 = len(X_train)
samples_10 = int(0.1 * len(X_train))
samples_1 = int(0.01 * len(X_train))

# Collect results on the learners
results = {}
for clf in [clf_A, clf_B, clf_C]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_1, samples_10, samples_100]):
        results[clf_name][i] = \
        train_predict(clf, samples, X_train, y_train, X_test, y_test)

# Run metrics visualization for the three supervised learning models chosen
vs.evaluate(results, accuracy, fscore)

In [ ]:
X_train.iloc[1]

In [ ]:
#plt.hist(labels[:300],bins=2)

In [ ]:
import visuals as vs
importances = RFmodel.feature_importances_

# Plot
vs.feature_plot(importances, X_train, y_train)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import r2_score
from sklearn.model_selection import ShuffleSplit

def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    # sklearn version 0.18: ShuffleSplit(n_splits=10, test_size=0.1, train_size=None, random_state=None)
    # sklearn versiin 0.17: ShuffleSplit(n, n_iter=10, test_size=0.1, train_size=None, random_state=None)
    cv_sets = ShuffleSplit(n_splits=10, test_size = 0.20, random_state=None)

    # TODO: Create a decision tree regressor object
    classifier = RandomForestClassifier()

    # TODO: Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    params = {'max_depth': range(1,10), 'n_estimators':[10,50,100,200]}

    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(accuracy_score)

    # TODO: Create the grid search cv object --> GridSearchCV()
    # Make sure to include the right parameters in the object:
    # (estimator, param_grid, scoring, cv) which have values 'regressor', 'params', 'scoring_fnc', and 'cv_sets' respectively.
    grid = GridSearchCV(classifier, params, scoring = scoring_fnc, cv = cv_sets)

    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

In [ ]:

reg = fit_model(X_train, y_train.values.ravel())

# Produce the value for 'max_depth'


print ("Parameter 'max_depth' is {} for the optimal model.".format(reg.get_params()['max_depth']))
print ("Parameter 'n_estimators' is {} for the optimal model.".format(reg.get_params()['n_estimators']))